In [84]:
import os
import numpy as np

import scipy

from scipy.optimize import curve_fit
from scipy.integrate import simpson
from scipy.optimize import minimize
import scipy.optimize as opt

# Set print options to display the full array
np.set_printoptions(threshold=np.inf)

new_dir_path = "/Users/mohammadmaster/Desktop"
os.chdir(new_dir_path)

from CHN_reader_F import Spectrum

#spectrum_instance = Spectrum('Co-60_SN_0813092.Chn')
#print(spectrum_instance.channel)
#print(len(spectrum_instance.channel))

directory = "/Users/mohammadmaster/Desktop/Montserrat sample backgrounds"

# Get a list of all files in the directory with extension .Chn
file_list = [f for f in os.listdir(directory) if f.endswith('.Chn')]

# Create a list of Spectrum instances
spectrum_list = []
for file_name in file_list:
    file_path = os.path.join(directory, file_name)
    spectrum_instance = Spectrum(file_path)
    spectrum_list.append(spectrum_instance)
print(len(spectrum_list))

#creates new array with only spectrums with less than 740 counts in the 3550 kev  and above range corresponds to channel 6375
new_spectrum_list = []

for spectrum_instance in spectrum_list:
    counts_above_3550 = sum(spectrum_instance.channel[6375:])
    if counts_above_3550 <= 740:
        new_spectrum_list.append(spectrum_instance)
        
spectrum_list = new_spectrum_list
print(len(spectrum_list))


#fits gaussian and shifts channel

# Define the Gaussian function
def gaussian(x, amplitude, mean, stddev, c):
    return amplitude * np.exp(-((x - mean) / stddev) ** 2) + c

def cost_function(params, x, y):
    return np.sum((gaussian(x, *params) - y) ** 2)

#channel detremined from energy line interested in, start and end of range determined by eye to ensure smooth fitting
spectral_range_start = 4670
spectral_line_channel = 4705
spectral_range_end = 4740
spectral_range_line = spectral_line_channel - spectral_range_start

epsilon = 0.013140455 # efficiency of the detector for the gamma ray energy
P = 1.0 # abundance of the gamma ray in the source material branching ratio

def redshift_spectrum(spectrum):
    # define the range of data to fit the gaussian to
    time = spectrum.livetime
    x_data = np.arange(spectral_range_start, spectral_range_end)
    y_data = spectrum.channel[x_data] 
    print(y_data)
    
    max_index = np.argmax(y_data)
    print(max_index)
    
    
    # initial guesses for the parameters of the gaussian function
    amplitude_guess = np.max(y_data)
    mean_guess = np.argmax(y_data)
    stddev_guess = 5.0
    c = 1.0

    initial_guess = [amplitude_guess, mean_guess, stddev_guess, c]

    # Minimize the cost function
    result = opt.minimize(cost_function, initial_guess, args=(np.arange(len(y_data)), y_data))

    # Get the optimized parameters
    params = result.x
    
    # shift the spectrum to center the peak on channel 1460 (channel 2634) 
    center_of_mass = np.sum(y_data * np.arange(len(y_data))) / np.sum(y_data)
    com_adjustment = center_of_mass - max_index
    print(center_of_mass)
    shift = spectral_line_channel - spectral_range_start - params[1] - com_adjustment
    print(shift)
    shifted_spectrum = np.roll(y_data, int(shift))
    
    # calculate the FWHM
    fwhm = 2.3548 * np.abs(params[2])
    
    print("std:", params[2])
    # calculate the area under the curve using FWHM
    area = np.sum(y_data[int(np.argmax(y_data) - fwhm) :int(np.argmax(y_data) + fwhm + 1)])
    
    # Calculate the RSS and estimate the standard deviation of the errors
    y_fit = gaussian(np.arange(len(y_data)), *params)
    rss = np.sum((y_data - y_fit)**2)
    stddev_error = np.sqrt(rss / (len(y_data) - len(params)))
    
    # Estimate the uncertainty in the area using error propagation
    dA_dh = np.sqrt(2 * np.pi) * fwhm / (2 * params[0])
    dA_dm = np.sqrt(2 * np.pi) * fwhm / (2.3548 * params[2]**2)
    dA_ds = np.sqrt(2 * np.pi) * (params[1] - np.argmax(y_data)) * fwhm / (2.3548 * params[2]**3)
    area_error = np.sqrt(dA_dh**2 * stddev_error**2 + dA_dm**2 * np.argmax(y_data) + dA_ds**2 * params[2]**2)

    

    return shifted_spectrum, params, area, area_error, time
    
# apply redshift_spectrum function to each spectrum in spectrum_list
shifted_spectrum_list = [redshift_spectrum(spectrum) for spectrum in spectrum_list]

background_activity_errors = []
background_params = []
background_activities = []
bg_durations = []
bg_spectra = []

# append areas and gaussian params to array and print values for each spectrum
for spectrum in shifted_spectrum_list:
    shifted_spectrum, params, area, area_error, time = spectrum
    activity = area / (epsilon * time * P) 
    background_activities.append(activity)
    background_activity_errors.append(area_error/ (epsilon * time * P))
    bg_durations.append(time)
    bg_spectra.append(shifted_spectrum)
    print("Spectrum around peak: ", shifted_spectrum)
    print("Amplitude:", params[0])
    print("Std Dev:", params[2])
    print("Area: ", area)
    print("Uncertainty:", area_error)
    print("Time:", time)
    
#average background areas and gaussian parameters
#avg_background_area = np.mean(background_areas)
#avg_background_params = np.mean(background_params, axis=0)

#create parameters for background gaussian centered on spectral line interested in within range
#background_gaussian = np.array([avg_background_params[0], spectral_range_line, avg_background_params[2]]) 


total_bg_duration = np.sum(bg_durations)
total_bg_spectrum = np.sum(bg_spectra, axis=0)

#bg_weights = bg_durations / np.sum(bg_durations)
#bg_avg = np.average(bg_spectra, axis=0, weights=bg_weights)

# Fit a Gaussian to the average background spectrum
p0 = [np.max(total_bg_spectrum), np.argmax(total_bg_spectrum), 5.0, 1.0]
params, _ = curve_fit(gaussian, np.arange(len(total_bg_spectrum)), total_bg_spectrum, p0=p0)

# Generate the Gaussian array for the background
bg_gaussian = gaussian(np.arange(len(total_bg_spectrum)), *params) / total_bg_duration
    
fwhm = 2.3548 * np.abs(params[2])
    
print("std:", params[2])
    # calculate the area under the curve using FWHM
area = np.sum(total_bg_spectrum[int(np.argmax(total_bg_spectrum) - fwhm) :int(np.argmax(total_bg_spectrum) + fwhm + 1)])
    

    # Calculate the RSS and estimate the standard deviation of the errors
y_fit = gaussian(np.arange(len(total_bg_spectrum)), *params)
rss = np.sum((total_bg_spectrum - y_fit)**2)
stddev_error = np.sqrt(rss / (len(total_bg_spectrum) - len(params)))
    
    # Estimate the uncertainty in the area using error propagation
dA_dh = np.sqrt(2 * np.pi) * fwhm / (2 * params[0])
dA_dm = np.sqrt(2 * np.pi) * fwhm / (2.3548 * params[2]**2)
dA_ds = np.sqrt(2 * np.pi) * (params[1] - np.argmax(total_bg_spectrum)) * fwhm / (2.3548 * params[2]**3)
dA_dc = fwhm / (2.3548 * np.sqrt(area))
area_error = np.sqrt(dA_dh**2 * stddev_error**2 + dA_dm**2 * np.argmax(total_bg_spectrum) + dA_ds**2 * params[2]**2 + dA_dc**2)



          New instance of class created
filename:/Users/mohammadmaster/Desktop/Montserrat sample backgrounds/2022-04-14_Background_A_063.Chn
The header is contained within the first 32 bytes: b'\xff\xff\x01\x00\x01\x0052 \xbf\x02\x00<\xbe\x02\x0017Apr2310847\x00\x00\x00@'
Measurement date/time: 08:47:52 17-Apr-23
realtime=3600.0
livetime=3595.44
channel offset=0
number of channels=16384
one hundred and two: b'\x9a\xff'
One hundred and two (little endian): -102
Calibration constant A= 0
Calibration constant B= 0
Calibration constant C= 0
Detector description:Trans-SPEC DX100-P 13310315
Sample description:Background
          New instance of class created
filename:/Users/mohammadmaster/Desktop/Montserrat sample backgrounds/2022-03-29_Background_A_188.Chn
The header is contained within the first 32 bytes: b'\xff\xff\x01\x00\x01\x0054 \xbf\x02\x00L\xbe\x02\x0006Apr2310557\x00\x00\x00@'
Measurement date/time: 05:57:54 06-Apr-23
realtime=3600.0
livetime=3595.76
channel offset=0
number of cha

In [80]:
bg_gaussian

array([0.00018956, 0.00018956, 0.00018956, 0.00018956, 0.00018956,
       0.00018956, 0.00018956, 0.00018956, 0.00018956, 0.00018956,
       0.00018956, 0.00018956, 0.00018956, 0.00018956, 0.00018956,
       0.00018956, 0.00018957, 0.00018957, 0.00018957, 0.00018958,
       0.0001896 , 0.00018965, 0.00018974, 0.00018993, 0.00019029,
       0.00019098, 0.00019225, 0.00019446, 0.00019823, 0.00020441,
       0.00021419, 0.0002291 , 0.00025103, 0.00028206, 0.00032433,
       0.00037967, 0.00044918, 0.00053281, 0.00062893, 0.00073405,
       0.00084283, 0.00094836, 0.00104288, 0.0011186 , 0.00116881,
       0.00118886, 0.00117684, 0.00113389, 0.00106403, 0.00097355,
       0.00087004, 0.00076138, 0.00065476, 0.00055597, 0.00046897,
       0.00039583, 0.00033699, 0.00029158, 0.00025791, 0.00023389,
       0.0002174 , 0.00020648, 0.00019952, 0.00019524, 0.00019269,
       0.00019124, 0.00019043, 0.00019   , 0.00018977, 0.00018966])

In [81]:
print(area)

81079


In [82]:
print(area_error)

3.4304657550722015


In [15]:
print(params)

[3511.4518451    25.80752129    5.7691545  1364.17964506]


In [83]:
total_bg_duration

3800460.6799999997